# LLm Features

## mask
To test weather mask's self attention has great impact over output logit

the mask value of itself won't have impact on itself.

That means the mask of i_th 1 in [1,1,...,1] don't have impact on P(x_i | x_i-1,...,x_0)

In [27]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer
from utils.utils import *

tokenizer = AutoTokenizer.from_pretrained("gpt2")
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
inference_model = GPT2LMHeadModel.from_pretrained("gpt2")
inference_model.to(device)
string = "The Project Gutenberg eBook of Complete Prose Works This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook."

with torch.no_grad():
        inputs = tokenizer(tokenizer.bos_token + string, return_tensors="pt")
        inputs.to(device)
        '''
        inputs["input_ids"]: tensor; shape of inputs["input_ids"] is [batchsize, token#]
        token_list: tensor; shape of token_list is [token#]
        '''
        token_list = inputs["input_ids"].squeeze(0)
        '''
        maxmium of i in this loop is *len(token_list) - 2*
        # in this token_list is len(token_list)
        maxmiun of index is len(token_list) - 1
        '''
        for i in range(1, len(token_list) - 1):
            current_token_index = token_list[i + 1]
            current_inputs = deepcopy(inputs)
            '''
            Tensor sliding:
                [:] all elements
                [n:] last n elements
                [:n] first n elements
            Shape of current_inputs["attention_mask"] and current_inputs["input_ids"]:
                [batchsize, i + 1]
            '''
            current_inputs["input_ids"] = current_inputs["input_ids"][:, :(i + 1)]
            current_inputs["attention_mask"] = current_inputs["attention_mask"][:, :(i + 1)]
            
            outputs = inference_model(**current_inputs, labels=current_inputs["input_ids"])
            '''
            Shape of logits:
                +[batchsize, tokens#, 50257(dict alpha/beta#)]
            '''
            logits = outputs.logits
            current_inputs["attention_mask"][:, -1] = 0
            outputs_modified = inference_model(**current_inputs, labels=current_inputs["input_ids"])
            logits_modified = outputs_modified.logits

            print(torch.equal(logits[0,-2], logits_modified[0,-2]))


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


## compress files main

In [28]:
from utils.utils import *
from utils.enwiki9_dataset import *
from mpmath import*


parser = argparse.ArgumentParser()
parser.add_argument("--maxdps", default=1600, type=int)
parser.add_argument("--device", type=str, default="gpu")
args = parser.parse_args()
torch.set_printoptions(precision=50)
mp.dps = args.maxdps

if args.device == "gpu":
    device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
elif args.device == "cpu":
    device = torch.device("cpu")

tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = GPT2LMHeadModel.from_pretrained("gpt2")
model = model.to(device) 

init_state = True
global_bound = ac_bound(torch.tensor(0.0), torch.tensor(1.0))

acc_count = 0
all_count = 0
global_compress_info = compress_info(0, 0)
with open('dataset/8813.txt', 'r', encoding='utf-8') as f:
    for line in f:
        stress_print(line)
        is_acc, current_compress_info = encoder_and_decoder(str_to_encoder = line, model=model, tokenizer=tokenizer, device = device)
        all_count += 1
        if is_acc:
            acc_count += 1 
            global_compress_info.update_compress_info(current_compress_info)
        pdb.set_trace()

usage: ipykernel_launcher.py [-h] [--maxdps MAXDPS] [--device DEVICE]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"ee3bcf82-1cc9-42ba-96b8-85f5e03383ac" --shell=9002 --transport="tcp" --iopub=9004 --f=/ubc/ece/home/rl/grads/qihangz/.local/share/jupyter/runtime/kernel-v2-415951vtD5Evj7MTfN.json


SystemExit: 2

## grammar

### slicing operation

In [ ]:
import torch
list_example = [
    [11,12],
    [21,22]
]
tensor_example = torch.tensor(list_example)
print(1,tensor_example)
print(2,tensor_example[:])
print(3,tensor_example[:,])
print(4,tensor_example[:, 1:])
print(5,tensor_example[:, :1])

### loop validate

In [ ]:
'''
for i in range(N):
maxmium of i is N -1
'''
for i in range(10):
    print(i)